In [ ]:
import os
import pickle as pk
from ipywidgets import interact
from copy import deepcopy

from tqdm import tqdm_notebook as tqdm

import xarray as xr
import numpy as np
import pandas as pd

import distributed

from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import matplotlib.pyplot as plt
import matplotlib.colors as mplc
import cartopy.crs as ccrs
import matplotlib.dates as mdates
from pandas.plotting import register_matplotlib_converters

In [ ]:
rnd = np.random.RandomState(42)

In [ ]:
DENSITY = 1000

In [ ]:
plt.style.use('paper')
plt.style.use('egu_journals')
register_matplotlib_converters()
plt.rcParams['text.usetex'] = False

In [ ]:
rotated_pole = ccrs.RotatedPole(pole_longitude=-171.0, pole_latitude=41.5)
plate_carree = ccrs.PlateCarree()

In [ ]:
cluster = distributed.LocalCluster(n_workers=40, threads_per_worker=1, memory_limit='2GB')
client = distributed.Client(cluster)
client

# Load data

In [ ]:
base_path = '/work/um0203/u300636/for2131/runs/da_enkf_for_soil/'
util_dir = '/work/um0203/u300636/for2131/runs/utilities'

## Nature run

In [ ]:
vr_h2o_path = os.path.join(base_path, '016', 'h2o_cleaned.nc')
vr_h2o = xr.open_dataset(vr_h2o_path)['H2OSOI'].squeeze(drop=True).isel(levsoi=4).chunk((1, 302, 267))

# SEKF

In [ ]:
da_h2o_sekf_vr_path = os.path.join(base_path, '023', 'juwels', 'da_offline_sekf_nature.nc')
da_h2o_sekf_vr = xr.open_dataarray(da_h2o_sekf_vr_path).squeeze(drop=True).isel(time=0).chunk((302, 267))

In [ ]:
da_h2o_sekf_ens_path = os.path.join(base_path, '023', 'juwels', 'da_offline_sekf_ens_nature.nc')
da_h2o_sekf_ens = xr.open_dataarray(da_h2o_sekf_ens_path).squeeze(drop=True).isel(time=0).chunk((302, 267))

In [ ]:
da_h2o_sekf_enkf_path = os.path.join(base_path, '023', 'juwels', 'da_offline_enkf_nature.nc')
da_h2o_sekf_enkf = xr.open_dataarray(da_h2o_sekf_enkf_path).squeeze(drop=True).isel(time=0).chunk((302, 267))

In [ ]:
da_h2o_sekf_3d_path = os.path.join(base_path, '023', 'juwels', 'da_offline_letkf_nature.nc')
da_h2o_sekf_3d = xr.open_dataarray(da_h2o_sekf_3d_path).squeeze(drop=True).isel(time=0).chunk((302, 267))

In [ ]:
da_h2o_sekf_h2o_path = os.path.join(base_path, '023', 'juwels', 'da_offline_enkf_h2o.nc')
da_h2o_sekf_h2o = xr.open_dataarray(da_h2o_sekf_h2o_path).squeeze(drop=True).isel(time=0).chunk((302, 267))

In [ ]:
da_h2o_sekf_ana_path = os.path.join(base_path, '023', 'juwels', 'da_offline_sekf.nc')
da_h2o_sekf_ana = xr.open_dataarray(da_h2o_sekf_ana_path).squeeze(drop=True).isel(time=0).chunk((302, 267))

In [ ]:
bg_h2o_path = os.path.join(base_path, '023', 'juwels', 'h2o_cleaned.nc')
bg_h2o = xr.open_dataarray(bg_h2o_path).squeeze(drop=True).sel(time='2015-07-31 00:00').isel(levsoi=4).chunk((302, 267))

In [ ]:
gain_orig_path = os.path.join(base_path, '023', 'juwels', 'gain_orig_sekf.nc')
gain_orig = xr.open_dataarray(gain_orig_path).squeeze(drop=True)

In [ ]:
gain_sekf_path = os.path.join(base_path, '023', 'juwels', 'da_offline_gain_sekf.nc')
gain_sekf = xr.open_dataarray(gain_sekf_path).squeeze(drop=True)

In [ ]:
gain_enkf_path = os.path.join(base_path, '023', 'juwels', 'da_offline_gain_enkf.nc')
gain_enkf = xr.open_dataarray(gain_enkf_path).squeeze(drop=True)

# Analyse error sources

In [ ]:
sekf_ana_err = da_h2o_sekf_ana - vr_h2o
sekf_vr_err = da_h2o_sekf_vr - vr_h2o
sekf_vr_ens_err = da_h2o_sekf_ens - vr_h2o
sekf_enkf_err = da_h2o_sekf_enkf - vr_h2o
sekf_3d_err = da_h2o_sekf_3d - vr_h2o
sekf_h2o_err = da_h2o_sekf_h2o - vr_h2o

In [ ]:
sekf_ana_inc = da_h2o_sekf_ana - bg_h2o
sekf_vr_inc = da_h2o_sekf_vr - bg_h2o
sekf_vr_ens_inc = da_h2o_sekf_ens - bg_h2o
sekf_enkf_inc = da_h2o_sekf_enkf - bg_h2o
sekf_3d_inc = da_h2o_sekf_3d - bg_h2o
sekf_h2o_inc = da_h2o_sekf_h2o - bg_h2o

In [ ]:
def calc_bounds(grid):
    grid_delta = np.mean(np.diff(grid))
    new_grid = np.concatenate((grid, [grid[-1]+grid_delta]), axis=0)
    new_grid -= grid_delta / 2
    return new_grid

def calc_pcolormesh_grid(*orig_grid):
    new_grids = tuple([calc_bounds(g) for g in orig_grid])
    return new_grids

In [ ]:
pcm_lon_lat = calc_pcolormesh_grid(sekf_ana_inc.lon.values, sekf_ana_inc.lat.values)
cmap = plt.get_cmap('BrBG')
norm = mplc.BoundaryNorm(np.linspace(-0.075, 0.075, 51), cmap.N, clip=True)

In [ ]:
fig, ax = plt.subplots()
cf = ax.pcolormesh(*pcm_lon_lat, sekf_ana_inc, vmin=-0.075, vmax=0.075, cmap=cmap)
fig.colorbar(cf)

In [ ]:
fig, ax = plt.subplots()
cf = ax.pcolormesh(*pcm_lon_lat, sekf_vr_inc, vmin=-0.075, vmax=0.075, cmap=cmap)
fig.colorbar(cf)

In [ ]:
fig, ax = plt.subplots()
cf = ax.pcolormesh(*pcm_lon_lat, sekf_3d_inc, vmin=-0.075, vmax=0.075, cmap=cmap)
fig.colorbar(cf)

In [ ]:
fig, ax = plt.subplots()
cf = ax.pcolormesh(*pcm_lon_lat, gain_orig.isel(time=0), vmin=-0.1, vmax=0.1, cmap=cmap)
fig.colorbar(cf)

In [ ]:
fig, ax = plt.subplots()
cf = ax.pcolormesh(*pcm_lon_lat, gain_sekf.isel(time=0), vmin=-0.1, vmax=0.1, cmap=cmap)
fig.colorbar(cf)